In [1]:
import pandas as pd

In [5]:
d = pd.read_csv ('/Users/azizbek/Documents/Projects/PowerLine/data/InsPLAD-fault/defect_supervised/glass-insulator/annotations/image_list_val.txt')

In [6]:
d.head()

,name category
0,missing-cap/Fotos 03-12-2020_DJI_0326_cadeia_i...
1,missing-cap/Fotos 03-12-2020_DJI_0334_cadeia_i...
2,missing-cap/Fotos 03-12-2020_DJI_0361_cadeia_i...
3,missing-cap/Fotos 03-12-2020_DJI_0363_cadeia_i...
4,missing-cap/Fotos 03-12-2020_DJI_0364_cadeia_i...


In [7]:
print(d.columns)

Index(['name category'], dtype='object')


In [11]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# === Config ===
DATA_DIR = "/Users/azizbek/Documents/Projects/PowerLine/data/InsPLAD-fault/defect_supervised/glass-insulator/train"
BATCH_SIZE = 8
NUM_EPOCHS = 20
IMAGE_SIZE = 224
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else
                      "cuda" if torch.cuda.is_available() else "cpu")

# === Data Loading ===
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# === Model ===
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classifier
model = model.to(DEVICE)

# === Loss & Optimizer ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    model.train()
    for imgs, labels in dataloader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {total_loss:.4f}")

# === Class Mapping ===
print("Class to index mapping:", dataset.class_to_idx)

/Users/azizbek/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/azizbek/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20, Loss: 35.2291
Epoch 2/20, Loss: 19.6432
Epoch 3/20, Loss: 8.0424
Epoch 4/20, Loss: 7.1335
Epoch 5/20, Loss: 5.5688
Epoch 6/20, Loss: 4.8206
Epoch 7/20, Loss: 3.5112
Epoch 8/20, Loss: 1.5944
Epoch 9/20, Loss: 6.9210
Epoch 10/20, Loss: 10.7436
Epoch 11/20, Loss: 10.5935
Epoch 12/20, Loss: 6.3708
Epoch 13/20, Loss: 11.7580
Epoch 14/20, Loss: 2.7617
Epoch 15/20, Loss: 4.3929
Epoch 16/20, Loss: 7.6988
Epoch 17/20, Loss: 9.6083
Epoch 18/20, Loss: 4.9320
Epoch 19/20, Loss: 3.9941
Epoch 20/20, Loss: 2.4931
Class to index mapping: {'good': 0, 'missing-cap': 1}


In [12]:
 # === Validation Dataset ===
DATA_DIR_VAL = "/Users/azizbek/Documents/Projects/PowerLine/data/InsPLAD-fault/defect_supervised/glass-insulator/val"
val_dataset = datasets.ImageFolder(DATA_DIR_VAL, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# === Validation Function ===
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"✅ Validation Accuracy: {accuracy:.2f}%")

# === Run Evaluation ===
evaluate(model, val_loader)

✅ Validation Accuracy: 83.05%


In [2]:
df = pd.read_csv("/Users/azizbek/Documents/Projects/PowerLine/dataset/image_labels_with_code.csv")

In [9]:
df_category2 = df[df["category_code"] == 3]
print(len(df_category2[df_category2["status"]==0]))
print(len(df_category2[df_category2["status"]==1]))
print(len(df_category2[df_category2["status"]==2]))
df_category2.shape


596
310
373


(1279, 4)